In [9]:
import dash
import dash_bootstrap_components as dbc
from dash import html, dcc, dash_table
import sqlite3
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import scipy.stats as stats
import plotly.express as px
import re
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
import plotly.figure_factory as ff

_conn = sqlite3.connect("airbnb_cartagena.sqlite")
df_attr = pd.read_sql_query("SELECT * FROM Attributes", _conn, dtype={"ID": str})
df_ts = pd.read_sql_query("SELECT * FROM TimeSeriesRaw", _conn, dtype={"ID": str})
df_ts_interp = pd.read_sql_query("SELECT * FROM TimeSeriesInterpolated", _conn, dtype={"ID": str})
_conn.close()

In [11]:
red = "#7e0d24"  # dark red color for plots

import re
dates = [col for col in df_ts.columns if re.fullmatch(r"\d{1,2}/\d{1,2}/\d{4}", col)]
df_ts_interp = df_ts_interp.dropna(subset=dates, how="any").reset_index(drop=True)

In [12]:
# 3.1 Compare Raw vs. Interpolated for Sample
def compare_raw_vs_interpolated():
    """
    Overlay raw (dashed) vs. interpolated (solid) daily price series
    for the 6 listings with the most missing entries —
    interpolated in red, raw in white (drawn on top).
    """
    # Pick the 6 listings with the most missing values
    df = df_ts.copy()
    df["missing_total"] = df[dates].isna().sum(axis=1)
    top3_missing = df.nlargest(3, "missing_total")["ID"].tolist()
    clean3_interp = (
        df_ts_interp
        .assign(missing_interp=lambda d: d[dates].isna().sum(axis=1))
        .query("missing_interp == 0").head(3)["ID"].tolist()
    )
    sample_ids = top3_missing + clean3_interp

    # Melt listings into long format
    raw = (
        df_ts
        .loc[df_ts["ID"].isin(sample_ids), ["ID","Name"] + dates]
        .melt(id_vars=["ID","Name"], var_name="Date", value_name="RawPrice")
    )
    raw["RawPrice"] = raw["RawPrice"].astype(float)
    interp = (
        df_ts_interp
        .loc[df_ts_interp["ID"].isin(sample_ids), ["ID","Name"] + dates]
        .melt(id_vars=["ID","Name"], var_name="Date", value_name="InterpPrice")
    )
    interp["InterpPrice"] = interp["InterpPrice"].astype(float)

    # Merge raw and interpolated dataframes
    df_cmp = pd.merge(raw, interp, on=["ID","Name","Date"])
    df_cmp["Date"] = pd.to_datetime(df_cmp["Date"], dayfirst=True, format="%d/%m/%Y")
    df_long = df_cmp.melt(
        id_vars=["ID","Name","Date"],
        value_vars=["RawPrice","InterpPrice"],
        var_name="Source",
        value_name="Price"
    ).replace({"Source": {"RawPrice": "Raw", "InterpPrice": "Interpolated"}})

    # Create the plot
    fig = px.line(
        df_long,
        x="Date", y="Price",
        color="Source",
        category_orders={"Source": ["Interpolated", "Raw"], "ID": sample_ids},
        line_dash="Source",
        line_dash_map={"Raw": "solid", "Interpolated": "dash"},
        color_discrete_map={"Interpolated": "#dddddd", "Raw": red},
        facet_col="ID", facet_col_wrap=2,
        facet_col_spacing=0.1,
        title="Raw vs. Interpolated Price Series",
        labels={"Price":"Price (USD/night)", "Date":"Date"},
        template="plotly_dark"
    )
    fig.update_traces(
        selector=dict(name="Raw"),
        line=dict(width=2),
        opacity=0.8
    )
    fig.update_layout(
        showlegend=True,
        height=300 * 3,
        legend_title_text="Series"
    )
    fig.for_each_annotation(lambda a: a.update(text=f"ID: {a.text.split('=')[1]}"))
    fig.update_xaxes(matches=None)
    fig.update_yaxes(matches=None)

    return fig

compare_raw_vs_interpolated()